In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch
print("And there was light!")
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import time
import h5py
from tqdm import tqdm
from sys import getsizeof
import gzip

And there was light!


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
print("Cuda available: ", torch.cuda.is_available())
print("Cuda devices: ", torch.cuda.device_count())
current_device = torch.cuda.current_device()
print("Current cuda device: ", current_device)
print("Current cuda device name: ", torch.cuda.get_device_name(current_device))
print(torch.cuda.get_device_properties(current_device))

Cuda available:  True
Cuda devices:  1
Current cuda device:  0
Current cuda device name:  NVIDIA L40S
_CudaDeviceProperties(name='NVIDIA L40S', major=8, minor=9, total_memory=45494MB, multi_processor_count=142, uuid=671aadf9-2757-b570-e349-dbf8f06407ba, L2_cache_size=96MB)


In [2]:
import pandas as pd
from pathlib import Path
from evo2 import Evo2

importing partial
importing hugging face


/home/jong505/conda/miniconda3/envs/evo2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


importing pkgutil
importing torch
importing typing
importing yaml
importing vortex1
importing vortex2


[05/31/25 08:32:51] WARNING  root - WARNING - Supported flash-attn versions are >= 2.1.1, <=       ]8;id=510539;file:///home/jong505/conda/miniconda3/envs/evo2/lib/python3.11/site-packages/transformer_engine/pytorch/attention.py\attention.py]8;;\:]8;id=16223;file:///home/jong505/conda/miniconda3/envs/evo2/lib/python3.11/site-packages/transformer_engine/pytorch/attention.py#164\164]8;;\
                             2.6.3. Found flash-attn 2.7.4.post1.                                                  

importing vortex3
importing vortex4
importing evo2 scoring
importing evo2 util


In [22]:
def create_embeddings(df: pd.DataFrame, l_name:str = "blocks.24.mlp.l3", axis=0, reduction="avg"):
    evo2_model = Evo2('evo2_1b_base', local_path="/home/jong505/models/evo2_1b_base.pt")

    x = df[["seq_x", "segment_id_x"]].copy()
    x.rename(columns={"seq_x": "seq", "segment_id_x": "segment_id"}, inplace=True)
    y = df[["seq_y", "segment_id_y"]].copy()
    y.rename(columns={"seq_y": "seq", "segment_id_y": "segment_id"}, inplace=True)
    df = pd.concat([x,y])
    df.drop_duplicates(subset="segment_id", inplace=True)

    results = {}

    for id, seq in tqdm(zip(df.segment_id.tolist(), df.seq.tolist()), total=df.shape[0]):
        if id in results: continue

        tok_x = torch.tensor(evo2_model.tokenizer.tokenize(seq), dtype=torch.int, ).unsqueeze(0).to('cuda:0')
        _, tens_x = evo2_model(tok_x, return_embeddings=True, layer_names=[l_name])
        x = tens_x[l_name].cpu().float().numpy()
        
        if reduction == "avg":
            results[id] = np.mean(x[0], axis=axis).tolist()
        elif reduction == "PCA":
            pca = PCA(n_components=1)
            results[id] = pca.fit_transform(x[0].T).flatten().tolist()
        elif reduction == "div3":
            split = np.array_split(x[0], 3)
            out = []
            for i in split:
                out += i.mean(axis=axis).tolist()
            results[id] = out
            
    df = pd.DataFrame(results.items(), columns=["segment_id","embedding"])
    return df

In [23]:
dataframes = ["data/aar_ath_bol_chi_cpa_tha/sm7_50000_test_seq.tsv", "data/aar_ath_bol_chi_cpa_tha/sm7_50000_train_seq.tsv", "data/aar_ath_bol_chi_cpa_tha/sm7_50000_val_seq.tsv"]
output = Path("data/aar_ath_bol_chi_cpa_tha/embeddings")
dataframes = [pd.read_csv(Path(df_path), sep="\t", header=0) for df_path in dataframes]
df = pd.concat(dataframes)
print(df.columns)
df = df.sample(n=3, random_state=42)
ems = create_embeddings(df, reduction="div3")

Index(['Unnamed: 0', 'genome_x', 'chr_x', 'len_profile_x', 'segment_id_x',
       'genome_y', 'chr_y', 'len_profile_y', 'segment_id_y', 'seq_x', 'seq_y',
       'multiplicon_id', 'similar'],
      dtype='object')
Loading model from /home/jong505/models/evo2_1b_base.pt...
Loading config from /home/jong505/conda/miniconda3/envs/evo2/lib/python3.11/site-packages/evo2/configs/evo2-1b-8k.yml...


100%|██████████| 25/25 [00:00<00:00, 543.10it/s]


Extra keys in state_dict: {'blocks.10.mixer.dense._extra_state', 'blocks.24.mixer.dense._extra_state', 'blocks.17.mixer.dense._extra_state', 'blocks.3.mixer.dense._extra_state', 'unembed.weight'}


/home/jong505/conda/miniconda3/envs/evo2/lib/python3.11/site-packages/transformer_engine/pytorch/module/base.py:630: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = to

In [26]:
ems["len"] = ems.embedding.apply(len)
ems.head()

,segment_id,embedding,len
0,28631,"[2.0824657553930592e-08, 1.0475597491677036e-0...",5760
1,27039,"[2.0236601727674497e-08, 1.028544147629873e-06...",5760
2,18577,"[1.9474136081498727e-08, 9.803535476748948e-07...",5760
3,28632,"[1.9990530120139738e-08, 1.0074290912598372e-0...",5760
4,26356,"[1.6620779419440623e-08, 8.777113293945149e-07...",5760


In [24]:
df = df_in.iloc[:2]
counter = 0
em_x = []
em_y = []
l_name = "blocks.24.mlp.l3"

for seq_x, seq_y in zip(df.seq_x, df.seq_y):

    tok_x = torch.tensor(evo2_model.tokenizer.tokenize(seq_x), dtype=torch.int, ).unsqueeze(0).to('cuda:0')
    _, tens_x = evo2_model(tok_x, return_embeddings=True, layer_names=[l_name])
    x = tens_x[l_name].cpu().float().numpy()

    tok_y = torch.tensor(evo2_model.tokenizer.tokenize(seq_y), dtype=torch.int, ).unsqueeze(0).to('cuda:0')
    _, tens_y = evo2_model(tok_y, return_embeddings=True, layer_names=[l_name])
    y = tens_y[l_name].cpu().float().numpy()

    s = time.time()
    pca = PCA(n_components=1)
    print("time: ", time.time()- s)
    print(x.shape)
    small = pca.fit_transform(x[0].T)
    print(small.flatten())

print(small)

# df_embeddings = df.drop(["seq_x", "seq_y"], axis=1)
# df_embeddings["em_x"] = em_x
# df_embeddings["em_y"] = em_y
# df_embeddings.to_csv("testing_embeddings.tsv", sep='\t', index=False)

time:  0.00014662742614746094
(1, 34380, 1920)
(1920, 1)
time:  7.200241088867188e-05
(1, 5137, 1920)
(1920, 1)
[[ 1.1243758e-06]
 [ 7.3430725e-05]
 [ 2.9602891e-06]
 ...
 [ 3.0072144e-06]
 [-1.3268522e-06]
 [-5.9386343e-06]]


In [28]:
print(small.flatten()[:10])

[ 1.1243758e-06  7.3430725e-05  2.9602891e-06  3.6436827e-06
  5.1114598e-06 -4.0481518e-06  5.4306977e-09 -6.9068547e-06
 -4.7741059e-06 -2.1640153e-06]


In [35]:
df_embeddings = pd.read_csv("testing_embeddings.tsv", sep='\t')
df_embeddings.em_x = df_embeddings.em_x.apply(eval)
df_embeddings.em_x[0][0]

float

In [5]:
train_test_path = Path("iadh_out/ath_bol_aar/train_test1.tsv")
df = pd.read_csv(train_test_path, sep="\t", header=0, index_col=0)

In [6]:
seq_sx = df.iloc[0]["seq_x"]
seq_sy = df.iloc[0]["seq_y"]
seq_dx = df[df["Similar"]==True].iloc[0]["seq_x"]
seq_dy = df[df["Similar"]==True].iloc[0]["seq_y"]

In [3]:
evo2_model = Evo2('evo2_1b_base')
print(evo2_model.model)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 2906.66it/s]


Found complete file in repo: evo2_1b_base.pt


100%|██████████| 25/25 [00:01<00:00, 12.52it/s]


Extra keys in state_dict: {'unembed.weight', 'blocks.3.mixer.dense._extra_state', 'blocks.17.mixer.dense._extra_state', 'blocks.24.mixer.dense._extra_state', 'blocks.10.mixer.dense._extra_state'}


/home/jong505/conda/miniconda3/envs/evo2/lib/python3.11/site-packages/transformer_engine/pytorch/module/base.py:630: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = to

StripedHyena(
  (embedding_layer): VocabParallelEmbedding(512, 1920)
  (blocks): ModuleList(
    (0): ParallelGatedConvBlock(
      (pre_norm): RMSNorm()
      (post_norm): RMSNorm()
      (filter): HyenaCascade()
      (projections): TELinear()
      (out_filter_dense): Linear(in_features=1920, out_features=1920, bias=True)
      (mlp): ParallelGatedMLP(
        (l1): Linear(in_features=1920, out_features=5120, bias=False)
        (l2): Linear(in_features=1920, out_features=5120, bias=False)
        (l3): Linear(in_features=5120, out_features=1920, bias=False)
      )
    )
    (1-2): 2 x ParallelGatedConvBlock(
      (pre_norm): RMSNorm()
      (post_norm): RMSNorm()
      (filter): HyenaCascade()
      (projections): TELinear()
      (out_filter_dense): Linear(in_features=1920, out_features=1920, bias=True)
      (mlp): ParallelGatedMLP(
        (act): Identity()
        (l1): Linear(in_features=1920, out_features=5120, bias=False)
        (l2): Linear(in_features=1920, out_features

/home/jong505/thesis/evo2/vortex/vortex/model/utils.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch_load(state, map_location=device)


In [14]:
evo2_model.tokenizer.vocab()

NotImplementedError: 

In [ ]:
# evo2_model = Evo2('evo2_1b_base')

input_ids = torch.tensor(
    evo2_model.tokenizer.tokenize_batch(seq_sx),
    dtype=torch.int,
).unsqueeze(0).to('cuda:0')

layer_name = 'blocks.24.mlp.l3'

outputs, embeddings = evo2_model(input_ids, return_embeddings=True, layer_names=[layer_name])

print('Embeddings shape: ', embeddings[layer_name].shape)

Embeddings shape:  torch.Size([1, 26728, 1920])
{'blocks.24.mlp.l3': tensor([[[ 2.5611e-08,  1.3039e-06,  5.9372e-08,  ...,  5.9139e-08,
          -1.6997e-08, -9.9186e-08],
         [ 2.5844e-08,  1.3262e-06,  5.9605e-08,  ...,  5.8208e-08,
          -1.5949e-08, -1.0058e-07],
         [ 2.5844e-08,  1.3337e-06,  5.9372e-08,  ...,  5.8440e-08,
          -1.5832e-08, -1.0105e-07],
         ...,
         [ 2.2701e-08,  1.2517e-06,  5.5181e-08,  ...,  5.3085e-08,
          -1.4028e-08, -9.3598e-08],
         [ 2.4680e-08,  1.3039e-06,  5.8673e-08,  ...,  5.5414e-08,
          -1.5018e-08, -9.8255e-08],
         [ 2.5611e-08,  1.2517e-06,  5.6578e-08,  ...,  5.3318e-08,
          -1.3621e-08, -9.3132e-08]]], device='cuda:0', dtype=torch.bfloat16)}


In [14]:
from tqdm import tqdm

In [ ]:
tokens = evo2_model.tokenizer.tokenize_batch(seqs)
counter = 0
layer_name = 'blocks.24.mlp.l3'
sim = []


for i, r in tqdm(df.iterrows(), total=df.shape[0]):
    seq_x = r["seq_x"]
    seq_y =  r["seq_y"]
    if len(seq_x) > len(seq_y):
        seq_y = seq_y+("0"* (len(seq_x)-len(seq_y)))
    elif len(seq_y) > len(seq_x):
        seq_x = seq_x+("0"* (len(seq_y)-len(seq_x)))
    
    tok_x = torch.tensor( evo2_model.tokenizer.tokenize(seq_x), dtype=torch.int, ).unsqueeze(0).to('cuda:0')
    tok_y = torch.tensor( evo2_model.tokenizer.tokenize(seq_y), dtype=torch.int, ).unsqueeze(0).to('cuda:0')

    
    _,tens_x = evo2_model(tok_x, return_embeddings=True, layer_names=[layer_name])
    x = tens_x[layer_name].cpu().float().numpy().flatten()
    # print(x.shape)

    _,tens_y = evo2_model(tok_y, return_embeddings=True, layer_names=[layer_name])
    y = tens_y[layer_name].cpu().float().numpy().flatten()
    # print(y.shape)

    sim.append(np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y)))
df["cosine_sim"] = sim

  0%|          | 0/1420 [00:00<?, ?it/s]

  1%|          | 16/1420 [01:25<2:05:39,  5.37s/it]


KeyboardInterrupt: 

In [66]:
small_df = df.sample(n=100)
small_df.value_counts("Similar")

Similar
False    56
True     44
Name: count, dtype: int64

In [ ]:
def main(df: pd.DataFrame):
    df[["seq_x", "seq_y"]] = df.apply(add_padding, axis=1)
    print(f"sim is {len(df.iloc[0]['seq_x']) == len(df.iloc[0]['seq_y'])}")
    sim = []

    for _, r in tqdm(df.iterrows(), total=df.shape[0]):
        print(torch.cuda.memory_allocated(0))
        print(len(seq_x))
        seq_x = r["seq_x"]
        seq_y =  r["seq_y"]
        
        tok_x = torch.tensor( evo2_model.tokenizer.tokenize(seq_x), dtype=torch.int, ).unsqueeze(0).to('cuda:0')
        tok_y = torch.tensor( evo2_model.tokenizer.tokenize(seq_y), dtype=torch.int, ).unsqueeze(0).to('cuda:0')
        
        _, tens_x = evo2_model(tok_x, return_embeddings=True, layer_names=[layer_name])
        x = tens_x[layer_name].cpu().float().numpy().ravel()

        _,tens_y = evo2_model(tok_y, return_embeddings=True, layer_names=[layer_name])
        y = tens_y[layer_name].cpu().float().numpy().ravel()

        sim.append(np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y)))
    df["cosine_sim"] = sim
    return df


def add_padding(row:pd.Series, pad_char:str= "0"):
    max_len = max(len(row["seq_x"]), len(row["seq_y"]))
    x = row['seq_x'] + pad_char*(max_len - len(row['seq_x']))
    y = row['seq_y'] + pad_char*(max_len - len(row['seq_y']))
    return pd.Series([x, y])

small_df_sim = main(small_df)

sim is True


  0%|          | 0/100 [00:00<?, ?it/s]

7487684608


  1%|          | 1/100 [00:03<04:56,  3.00s/it]

7629438976


...

7896335872


 10%|█         | 10/100 [00:43<06:24,  4.27s/it]

7669262848


 10%|█         | 10/100 [00:50<07:33,  5.04s/it]


KeyboardInterrupt: 